<a href="https://colab.research.google.com/github/juniorskg/recommandation_system/blob/main/book_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
!pip install summarytools


In [105]:
import pandas as pd
from operator import add
from functools import reduce
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from summarytools import dfSummary


# function to fill null value with the best central measure  between mean and median

In [106]:
def central_tendancy_calculator(a,column):
#replacing dollar sign and converting to float if necessary
  try:
    # Remove dollar signs and commas, then convert to float
    a[column] = a[column].str.replace(r'[$,]', '', regex=True).astype(float)
  except Exception as e:
    print(f"Error converting column {column}: {e}")
# remove missing value to make calculation
  dataset=pd.DataFrame()
  dataset[column]=a[column].dropna()



#calculate mean
  mean=reduce(add,dataset[column] )/len(dataset[column] )

#calculate interquatile ranges to check the outliers


  Q1 = dataset[column] .quantile(0.25)# Calculate Q1 (25th percentile)
  Q3 = dataset[column] .quantile(0.75)# CalculateQ3 (75th percentile)

  # Calculate the IQR
  IQR = Q3 - Q1

  # Determine the lower and upper bounds for outliers
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # Identify outliers

  outliers= (dataset[column]  < lower_bound) | (dataset[column]  > upper_bound)
# calculate median (median is better when the dataset have outliers)
  median= dataset[column].median()
# calculate the mode
  mode=dataset[column].mode()
# calculate skewness
  Skewness=dataset[column].skew()
#calculate variance
  variance=dataset[column].var()
# print result
  # print mean
  print(f"the mean is {mean}")
  # print meddian
  print(f"the median is {median}")
  # print mode
  print(f"the mode is {mode}")
  # print variance
  print(f"the variance is {variance}")
  # print Skewness
  print(f"the skewness is {Skewness}")
  # Print only the outliers
  print("Outliers using IQR method:")
  print( dataset[outliers].count() )

  if int(dataset[outliers].count())==0 and (Skewness>=-0.5 or Skewness<=0.5) :
    print("null value will be fill with the mean")
    a[column].fillna(mean, inplace=True)
  else:
     print("null value will be fill with the median")
     a[column].fillna(median, inplace=True)



# function to fill null value with mode

In [107]:
def mode_calculator(a,column):
    # remove missing value to make calculation
  dataset=pd.DataFrame()
  dataset[column]=a[column].dropna()
  # calculate the mode
  mode=dataset[column].mode()
   # print mode
  print(f"the mode is {mode}")
  a[column].fillna(str(mode), inplace=True)

# import dataset

In [108]:
book=pd.read_csv("Books.csv",sep=",")
rating=pd.read_csv("Ratings.csv",sep=",")

<ipython-input-108-b5700aee6d81>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book=pd.read_csv("Books.csv",sep=",")


# **book data set**

In [109]:
book.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


remove useless data

In [110]:
book=book.drop(["Image-URL-S","Image-URL-M","Image-URL-L"],axis=1)
book.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


check  data type

In [111]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


check null data

In [112]:
book.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
dtype: int64

# ***ISBN***

check if ISBN is unique

In [113]:
print(book["ISBN"].is_unique)

True


# book authors

fill null values

In [114]:
mode_calculator(book,"Book-Author")


the mode is 0    Agatha Christie
Name: Book-Author, dtype: object


# Year-Of-Publication

put null for every values that can not be numeric

In [115]:
book["Year-Of-Publication"]=pd.to_numeric(book["Year-Of-Publication"],errors='coerce')

remove invalid year (after 2024 or before 1455 which the year where The first major book was  published and made widely available to the general public using the printing press)

In [116]:
book["Year-Of-Publication"]=book["Year-Of-Publication"].where(book["Year-Of-Publication"]>1454,np.nan )
book["Year-Of-Publication"]=book["Year-Of-Publication"].where(book["Year-Of-Publication"]<2025,np.nan )



fill null values

In [117]:
central_tendancy_calculator(book,"Year-Of-Publication")



Error converting column Year-Of-Publication: Can only use .str accessor with string values!
the mean is 1993.6925710000937
the median is 1996.0
the mode is 0    2002.0
Name: Year-Of-Publication, dtype: float64
the variance is 66.39437819407586
the skewness is -1.6376506316492516
Outliers using IQR method:
Year-Of-Publication    4463
dtype: int64
null value will be fill with the median


<ipython-input-106-c544a2f39e50>:56: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if int(dataset[outliers].count())==0 and (Skewness>=-0.5 or Skewness<=0.5) :


convert Year-Of-Publication to int

In [118]:

book["Year-Of-Publication"]=book["Year-Of-Publication"].astype(int)

# Publisher

fill null values

In [119]:

mode_calculator(book,"Publisher")

the mode is 0    Harlequin
Name: Publisher, dtype: object


# **RATING Data set**

In [120]:
rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


verify data type

In [121]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


remove rating greater than 10 or smaller than 0

In [122]:
rating["Book-Rating"]=rating["Book-Rating"].where(rating["Book-Rating"]>=0,np.nan )
rating["Book-Rating"]=rating["Book-Rating"].where(rating["Book-Rating"]<=10,np.nan )



verify null values

In [123]:
rating.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

remove user id and map each book to its mean rating

In [124]:
rating=rating[["ISBN","Book-Rating"]]

rating= rating.groupby("ISBN").agg({"Book-Rating": "mean"}).reset_index()


# **merge dataset**

In [125]:
df=pd.merge(book, rating, on='ISBN')
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,0.000000
...,...,...,...,...,...,...
270146,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),7.000000
270147,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,4.000000
270148,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,0.000000
270149,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,0.000000


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270151 entries, 0 to 270150
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 270151 non-null  object 
 1   Book-Title           270151 non-null  object 
 2   Book-Author          270151 non-null  object 
 3   Year-Of-Publication  270151 non-null  int64  
 4   Publisher            270151 non-null  object 
 5   Book-Rating          270151 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 12.4+ MB


check if there are null values

In [127]:
df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Book-Rating            0
dtype: int64

check again if ISBN is unique

In [128]:
df["ISBN"].is_unique

True

collect the book released after 2003

In [129]:

df = df[(df["Year-Of-Publication"]>2003) ]

check the number of books

In [130]:
df["Year-Of-Publication"].count()

5810

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5810 entries, 12 to 270148
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ISBN                 5810 non-null   object 
 1   Book-Title           5810 non-null   object 
 2   Book-Author          5810 non-null   object 
 3   Year-Of-Publication  5810 non-null   int64  
 4   Publisher            5810 non-null   object 
 5   Book-Rating          5810 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 317.7+ KB


creat original dataset to keep the main data as we are going to perform modifications on df

In [132]:
original_df=df

In [133]:
dfSummary(original_df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,ISBN[object],1. 08878417402. 14000409653. 03754211654. 04402421695. 03129279246. 18545979147. 06898707528. 074900651X9. 006009005710. 006029519811. other,"1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)5,800 (99.8%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADLklEQVR4nO3cwY3TYBRG0ecogmFja2QpJUAPKYItvdOFNzCswoIeriVzTgX/4kpPkfxleb1eA4Xb2Q/g/yE2MmIjIzYyYiMjNjJiIyM2MmIjIzYyt2VZvi3L8nb2Q7i+28x8nZnPZz+E63NGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI3MbWZ+zsyfsx/C9S3+xYiKM0pGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kDF4IeOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEZGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKBmxkREbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYy1lVkrKvIOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKJllZr7MzKezH8L13fd9/76u6/vZD+H67uu6vj+fz9/btn2c/Riu6ziOt/vMzLZtH4/H49fZD+La/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjc5/59/nH2Q/h2o7jeFv2ff/h40kKPgsnY11Fxg8EMmIjIzYyYiMjNjJiIyM2MmIjIzYyfwFj50uCzHpkKgAAAABJRU5ErkJggg=="">",0(0.0%)
2,Book-Title[object],1. The Last Juror2. My Life3. 3rd Degree4. Sanctuary5. Little Women6. Flame Of Recca (Flame Of Recca7. Left for Dead8. Can You Keep a Secret?9. Lawless10. Fake (Fake)11. other,"4 (0.1%)3 (0.1%)3 (0.1%)3 (0.1%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)5,785 (99.6%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADQklEQVR4nO3cwWkbYRRG0TdCJPZGQghUQtKDisg2HauINGGkReJ44yyCS8gdmJxTwb+48BDMp+X9/X2gsFv7Afw/xEZGbGTERkZsZMRGRmxkxEZGbGTERma3LMvXZVme1n4I27ebmS8z83nth7B9zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkdnNzI+Z+b32Q9i+xb8YUXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhklIzYyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsa6iox1FRlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhkls8zM88x8WvshbN7b/nw+fzscDqe1X8K2PR6Pl/3hcDhdr9dfx+Pxde0HsU33+/3pdrud9jMzx+Px9XK5/Fz7UWzasx8IZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZPYzfz8BWfshbNdHX8v5fP7u40n+tcfj8eKzcCpv1lVk/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiPzB0WpV2yec8bJAAAAAElFTkSuQmCC"">",0(0.0%)
3,Book-Author[object],1. Agatha Christie2. Nora Roberts3. Diana Palmer4. Stephen King5. William Shakespeare6. John Sandford7. Meg Cabot8. Quino9. J. D. Robb10. James Patterson11. other,"25 (0.4%)21 (0.4%)12 (0.2%)12 (0.2%)11 (0.2%)11 (0.2%)11 (0.2%)11 (0.2%)9 (0.2%)8 (0.1%)5,679 (97.7%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADmUlEQVR4nO3cMU4kZxRG0VeohYekWq2WWIK9BxYxi51F9CK8B4SowIwnKQdjW5Zjzy2pfU5E+AdXPED9sez7PlB4OPoB/H+IjYzYyIiNjNjIiI2M2MiIjYzYyCwz8zQzs+/7x8Fv4c6drtfr55mZZVm+CI4f6WFd18u6rpeZeTz6Mdw3P7ORERsZsZERGxmxkREbGbGRERsZsZE

set ISBN as the index

In [134]:
original_df= original_df.rename(index=dict(zip(original_df.index, df["ISBN"])))

In [135]:
original_df=original_df.drop("ISBN",axis=1)

# **encode the data**

convert non-numerical data to tfidf

In [136]:
Tfidf_Vectorizer_Book_Title=TfidfVectorizer()
Tfidf_Vectorizer_Book_Author=TfidfVectorizer()
Tfidf_Vectorizer_Publisher=TfidfVectorizer()

Tfidf_of_Book_Title=Tfidf_Vectorizer_Book_Title.fit_transform(df['Book-Title'])
Tfidf_of_Book_Author=Tfidf_Vectorizer_Book_Author.fit_transform(df['Book-Author'])
Tfidf_of_Publisher=Tfidf_Vectorizer_Publisher.fit_transform(df["Publisher"])



put them as dataframe

In [137]:

Tfidf_of_Book_Title_df= pd.DataFrame(Tfidf_of_Book_Title.toarray(), columns=Tfidf_Vectorizer_Book_Title.get_feature_names_out())
Tfidf_of_Book_Author_df = pd.DataFrame(Tfidf_of_Book_Author.toarray(), columns=Tfidf_Vectorizer_Book_Author.get_feature_names_out())
Tfidf_of_Publisher_df = pd.DataFrame(Tfidf_of_Publisher.toarray(), columns=Tfidf_Vectorizer_Publisher.get_feature_names_out())

normalize numerical data

In [138]:


scaler = MinMaxScaler()
df[["Year-Of-Publication","Book-Rating"]] = scaler.fit_transform(df[['Year-Of-Publication',"Book-Rating"]])

<ipython-input-138-2c585874342e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Year-Of-Publication","Book-Rating"]] = scaler.fit_transform(df[['Year-Of-Publication',"Book-Rating"]])


In [139]:
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Book-Rating
12,0887841740,The Middle Stories,Sheila Heti,0.0,House of Anansi Press,0.250000
26,0971880107,Wild Animus,Rich Shapero,0.0,Too Far,0.101958
117,0553584383,Dead Aim,IRIS JOHANSEN,0.0,Bantam Books,0.310909
120,0316735736,All He Ever Wanted: A Novel,Anita Shreve,0.0,Back Bay Books,0.396875
243,140003180X,The Kalahari Typing School for Men (No. 1 Ladi...,ALEXANDER MCCALL SMITH,0.0,Anchor,0.329310
...,...,...,...,...,...,...
270087,1578567475,Hyperlinkz Book 1 : Digital Disaster (Hyperlinkz),ROBERT ELMER,0.0,WaterBrook Press,0.700000
270116,0441011799,Singularity Sky,Charles Stross,0.0,Ace,0.000000
270140,0520242335,Strong Democracy : Participatory Politics for ...,Benjamin R. Barber,0.0,University of California Press,0.000000
270143,1845170423,Cocktail Classics,David Biggs,0.0,Connaught,0.700000


In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5810 entries, 12 to 270148
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ISBN                 5810 non-null   object 
 1   Book-Title           5810 non-null   object 
 2   Book-Author          5810 non-null   object 
 3   Year-Of-Publication  5810 non-null   float64
 4   Publisher            5810 non-null   object 
 5   Book-Rating          5810 non-null   float64
dtypes: float64(2), object(4)
memory usage: 446.8+ KB


merge TFIDF data with normalise numerical data

In [141]:
years_df = df[["Year-Of-Publication"]].reset_index(drop=True)
rat=df[["Book-Rating"]].reset_index(drop=True)

df1 = pd.concat([rat,years_df, Tfidf_of_Book_Title_df,Tfidf_of_Book_Author_df, Tfidf_of_Publisher_df], axis=1)

In [142]:
df1

,Book-Rating,Year-Of-Publication,000,007,06,10,100,1001,1002,1003,...,xlibris,xulon,yale,ynr,york,young,zagat,zebra,zonderkidz,zondervan
0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.101958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.310909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.396875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.329310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5805,0.700000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5806,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5807,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5808,0.700000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#  calculate the cosine similarity between each books

In [143]:

similarity_matrix = cosine_similarity(df1)
similarity_df = pd.DataFrame(similarity_matrix)

In [144]:
similarity_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5800,5801,5802,5803,5804,5805,5806,5807,5808,5809
0,1.000000,0.008395,0.025240,0.031907,0.032194,0.055698,0.037376,0.019608,0.025297,0.040887,...,0.000000,0.004418,0.089092,0.065378,0.046761,0.093539,0.0,0.103396,0.053529,0.017314
1,0.008395,1.000000,0.010382,0.013125,0.010976,0.022911,0.015374,0.008066,0.005629,0.012375,...,0.000000,0.000000,0.019235,0.024641,0.019235,0.022019,0.0,0.000000,0.022019,0.000000
2,0.025240,0.010382,1.000000,0.072510,0.033000,0.068885,0.046225,0.024250,0.016923,0.070352,...,0.053463,0.000000,0.057832,0.165208,0.057832,0.066202,0.0,0.000000,0.066202,0.048015
3,0.031907,0.013125,0.072510,1.000000,0.041717,0.087080,0.058435,0.030655,0.021393,0.364661,...,0.029919,0.000000,0.073108,0.093653,0.073108,0.083689,0.0,0.000000,0.083689,0.000000
4,0.032194,0.010976,0.033000,0.041717,1.000000,0.072823,0.048868,0.025636,0.023222,0.044293,...,0.000000,0.002028,0.063548,0.080834,0.061139,0.069987,0.0,0.014629,0.069987,0.007949


set ISBN as the index

In [145]:
similarity_df = similarity_df.rename(index=dict(zip(similarity_df.index, df["ISBN"])))

In [146]:
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,5800,5801,5802,5803,5804,5805,5806,5807,5808,5809
0887841740,1.000000,0.008395,0.025240,0.031907,0.032194,0.055698,0.037376,0.019608,0.025297,0.040887,...,0.000000,0.004418,0.089092,0.065378,0.046761,0.093539,0.0,0.103396,0.053529,0.017314
0971880107,0.008395,1.000000,0.010382,0.013125,0.010976,0.022911,0.015374,0.008066,0.005629,0.012375,...,0.000000,0.000000,0.019235,0.024641,0.019235,0.022019,0.0,0.000000,0.022019,0.000000
0553584383,0.025240,0.010382,1.000000,0.072510,0.033000,0.068885,0.046225,0.024250,0.016923,0.070352,...,0.053463,0.000000,0.057832,0.165208,0.057832,0.066202,0.0,0.000000,0.066202,0.048015
0316735736,0.031907,0.013125,0.072510,1.000000,0.041717,0.087080,0.058435,0.030655,0.021393,0.364661,...,0.029919,0.000000,0.073108,0.093653,0.073108,0.083689,0.0,0.000000,0.083689,0.000000
140003180X,0.032194,0.010976,0.033000,0.041717,1.000000,0.072823,0.048868,0.025636,0.023222,0.044293,...,0.000000,0.002028,0.063548,0.080834,0.061139,0.069987,0.0,0.014629,0.069987,0.007949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578567475,0.093539,0.022019,0.066202,0.083689,0.069987,0.146090,0.098033,0.051429,0.035890,0.078908,...,0.000000,0.000000,0.184703,0.180784,0.122650,1.000000,0.0,0.036799,0.140401,0.000000
0441011799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
0520242335,0.103396,0.000000,0.000000,0.000000,0.014629,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.034105,0.000000,0.000000,0.036799,0.0,1.000000,0.000000,0.000000
1845170423,0.053529,0.022019,0.066202,0.083689,0.069987,0.146090,0.098033,0.051429,0.035890,0.078908,...,0.000000,0.000000,0.122650,0.157118,0.122650,0.140401,0.0,0.000000,1.000000,0.000000


# **recomandation function**

function that recommand the  most similar books tothe one you have

In [147]:
def recommendations(ISBN, original_df=original_df,cosine_sim=similarity_df):
    # ask the number of  recomandation you want
    no_recom= int(input(" how many recommandation do you need"))
    # take the similarity of that book with others
    isbn_cosine_sim=cosine_sim.loc[ISBN]
    # sort it in descending order
    isbn_cosine_sim = isbn_cosine_sim.sort_values(ascending=False)
    # take the top (no_of recom) similar book
    top=isbn_cosine_sim.head(no_recom).index
    print(top)
    indexes = list(map(int,top))
    selected_rows = original_df.iloc[indexes]
    selected_rows
    return selected_rows

In [148]:
#original_df.loc["006008667X"]

test the function

In [149]:
a=recommendations("006008667X")


 how many recommandation do you need5
Index([5809, 1359, 1550, 4394, 3487], dtype='int64')


In [150]:
a

,Book-Title,Book-Author,Year-Of-Publication,Publisher,Book-Rating
006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,0.0
0060563419,The Secrets of Jin-shei : A Novel,Alma Alexander,2004,HarperSanFrancisco,1.5
0060733993,"The Great Failure : A Bartender, A Monk, and M...",Natalie Goldberg,2004,HarperSanFrancisco,2.5
0060727667,"Home to Harmony (Gulley, Philip)",Philip Gulley,2004,HarperSanFrancisco,0.0
0062517678,"The Big Bang, The Buddha, and the Baby Boom",Wes Nisker,2004,HarperSanFrancisco,6.0
